<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Visualize-WebGL-Results-in-Python" data-toc-modified-id="Visualize-WebGL-Results-in-Python-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Visualize WebGL Results in Python</a></span></li></ul></li></ul></li><li><span><a href="#Compare-Accuracy-Numerical-(WebGL)-and-Analytic" data-toc-modified-id="Compare-Accuracy-Numerical-(WebGL)-and-Analytic-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Compare Accuracy Numerical (WebGL) and Analytic</a></span></li></ul></div>

In [81]:
import sympy as sp
import numpy as np
np.set_printoptions(threshold=np.inf) #When we want notebook to display everything

In [82]:
import ipyvolume as ipv  # Does not come with default anaconda #Used to show 3D graphs
import ipywidgets as widgets

In [2]:
p_x, p_y, p_z, th, phi, = sp.symbols(r'p_x p_y p_z \theta \phi')


In [16]:
A = sp.Matrix([[1,0,0,p_x],
               [0,1,0,p_y],
               [0,0,1,-p_z],
               [0,0,0,1]])
B = sp.Matrix([[sp.cos(th),0,sp.sin(th),0],
               [0,1,0,0],
               [-sp.sin(th),0,sp.cos(th),0],
               [0,0,0,1]])
C = sp.Matrix([[1,0,0,0],
              [0,sp.cos(phi),-sp.sin(phi),0],
              [0,sp.sin(phi),sp.cos(phi),0],
              [0,0,0,1]])

D = sp.Matrix([[1,0,0,-p_x],
               [0,1,0,-p_y],
               [0,0,1, p_z],
               [0,0,0,1]])

In [19]:
A*B*C*D

Matrix([
[ cos(\theta), sin(\phi)*sin(\theta), sin(\theta)*cos(\phi), -p_x*cos(\theta) + p_x - p_y*sin(\phi)*sin(\theta) + p_z*sin(\theta)*cos(\phi)],
[           0,             cos(\phi),            -sin(\phi),                                           -p_y*cos(\phi) + p_y - p_z*sin(\phi)],
[-sin(\theta), sin(\phi)*cos(\theta), cos(\phi)*cos(\theta),  p_x*sin(\theta) - p_y*sin(\phi)*cos(\theta) + p_z*cos(\phi)*cos(\theta) - p_z],
[           0,                     0,                     0,                                                                              1]])

In [83]:
def GetPoints(x,y):
    '''
    input
    x: np array
        the x positions of the grid points
        
    y: np array
        the y positions of the grid points
        
    returns
    np array
        with the position of the points going from bottom left to bottom right, then the next layer
    
    '''
    mesh = np.meshgrid(x,y)
    x_parts = np.reshape(mesh[0],(1,-1))[0]
    y_parts = np.reshape(mesh[1],(1,-1))[0]
    points = np.array([[x_parts[i],y_parts[i]] for i in range(len(x_parts))])
    return np.reshape(points,(1,-1))[0]

##Tests
actual = GetPoints([0,1,2],[0,1])
expected = np.array([0,0,1,0,2,0,0,1,1,1,2,1])
assert max(abs(actual-expected))==0

In [84]:
def GetIndices(x, y):
    '''
    input
    x: np array
        the x positions of the grid points

    y: np array
        the y positions of the grid points

    returns
    np array
        with the indices of all the linking points to make 3 vertex shapes

    '''
    w = len(x) #width
    def get_pos(x_pos, y_pos):
        return w*y_pos + x_pos #get the index position
    
    def get_points(x_pos, y_pos):
        return np.array([get_pos(x_pos,y_pos), get_pos(x_pos+1,y_pos), get_pos(x_pos,y_pos+1),
                         get_pos(x_pos+1,y_pos), get_pos(x_pos,y_pos+1), get_pos(x_pos+1,y_pos+1)])
    
    indices = []
    for y_pos in range(len(y)-1):
        for x_pos in range(len(x)-1):
            indices.append(get_points(x_pos, y_pos))
            
    return np.reshape(np.array(indices),(1,-1))[0]

##Tests
actual = GetIndices([0,1,2],[0,1])
expected = np.array([0,1,3,
                     1,3,4,
                     1,2,4,
                     2,4,5])
assert max(abs(actual-expected)) == 0

In [85]:
x = np.linspace(-2,2,101)#41)
y = np.linspace(-2,2,101)#41)
GetPoints(x,y)

array([-2.  , -2.  , -1.96, -2.  , -1.92, -2.  , -1.88, -2.  , -1.84,
       -2.  , -1.8 , -2.  , -1.76, -2.  , -1.72, -2.  , -1.68, -2.  ,
       -1.64, -2.  , -1.6 , -2.  , -1.56, -2.  , -1.52, -2.  , -1.48,
       -2.  , -1.44, -2.  , -1.4 , -2.  , -1.36, -2.  , -1.32, -2.  ,
       -1.28, -2.  , -1.24, -2.  , -1.2 , -2.  , -1.16, -2.  , -1.12,
       -2.  , -1.08, -2.  , -1.04, -2.  , -1.  , -2.  , -0.96, -2.  ,
       -0.92, -2.  , -0.88, -2.  , -0.84, -2.  , -0.8 , -2.  , -0.76,
       -2.  , -0.72, -2.  , -0.68, -2.  , -0.64, -2.  , -0.6 , -2.  ,
       -0.56, -2.  , -0.52, -2.  , -0.48, -2.  , -0.44, -2.  , -0.4 ,
       -2.  , -0.36, -2.  , -0.32, -2.  , -0.28, -2.  , -0.24, -2.  ,
       -0.2 , -2.  , -0.16, -2.  , -0.12, -2.  , -0.08, -2.  , -0.04,
       -2.  ,  0.  , -2.  ,  0.04, -2.  ,  0.08, -2.  ,  0.12, -2.  ,
        0.16, -2.  ,  0.2 , -2.  ,  0.24, -2.  ,  0.28, -2.  ,  0.32,
       -2.  ,  0.36, -2.  ,  0.4 , -2.  ,  0.44, -2.  ,  0.48, -2.  ,
        0.52, -2.  ,

In [86]:
GetIndices(x,y)

array([    0,     1,   101,     1,   101,   102,     1,     2,   102,
           2,   102,   103,     2,     3,   103,     3,   103,   104,
           3,     4,   104,     4,   104,   105,     4,     5,   105,
           5,   105,   106,     5,     6,   106,     6,   106,   107,
           6,     7,   107,     7,   107,   108,     7,     8,   108,
           8,   108,   109,     8,     9,   109,     9,   109,   110,
           9,    10,   110,    10,   110,   111,    10,    11,   111,
          11,   111,   112,    11,    12,   112,    12,   112,   113,
          12,    13,   113,    13,   113,   114,    13,    14,   114,
          14,   114,   115,    14,    15,   115,    15,   115,   116,
          15,    16,   116,    16,   116,   117,    16,    17,   117,
          17,   117,   118,    17,    18,   118,    18,   118,   119,
          18,    19,   119,    19,   119,   120,    19,    20,   120,
          20,   120,   121,    20,    21,   121,    21,   121,   122,
          21,    22,

### Visualize WebGL Results in Python

In [1]:
def read_file(file_name):
    f = open(file_name, "r")
    my_string_data = f.read().splitlines()
    my_Z = []
    for row_string_data in my_string_data:
        row_string_list = row_string_data.split(',')
        my_time_frame_row = []
        for element_index in range(len(row_string_list)-1):
            my_time_frame_row.append(float(row_string_list[element_index]))
        my_Z.append(np.array([my_time_frame_row]))
    return np.array(my_Z)

In [2]:
Z_Num = read_file("E:\Downloads\Test (17).txt")


Num_xy = len(Z_Num[0][0])**0.5
print(Num_xy)
a = np.linspace(0, 1, num=int(Num_xy), endpoint=True)
b = np.linspace(0, 1, num=int(Num_xy), endpoint=True)
U, V = np.meshgrid(a, b)
X = U
Y = V

ipv.figure()
s = ipv.plot_surface(X, Y, Z_Num, color="orange")
#w = ipv.plot_wireframe(X, Y, Z, color="red")
ipv.animation_control(s, add = True, interval=200)#, sequence_length=2)
#mylink = widgets.link((s, 'sequence_index'), (w, 'sequence_index'))
ipv.show()

22.0


NameError: name 'ipv' is not defined

# Compare Accuracy Numerical (WebGL) and Analytic 

In [138]:
#Analytic Solution
MAX_N = 100

def Q_n(n_x,n_y):
    return np.pi*(n_x**2+n_y**2)**0.5

def B_n(n_x, n_y):
    return 4*(-1)**(n_x+n_y)/(n_x*n_y*np.pi**2)

def u(x,y,t):
    total = 0
    for n_x in range(1, MAX_N):
        for n_y in range(1, MAX_N):
            total += B_n(n_x,n_y)*np.sin(n_x*np.pi*x)*np.sin(n_y*np.pi*y)*np.cos(Q_n(n_x,n_y)*t)
    return total

In [101]:
#Put time and run commands
X = U
Y = V  # The y-axis is the wrong way around
#Z = np.array([[np.real(u(x,y)) for x in a] for y in b])
Z_Ana = np.array([[[u(x,y,t) for x in a] for y in b] for t in [0,1,2,3,4.4,16]])#np.linspace(0,5,10)])

ipv.figure()
s = ipv.plot_surface(X, Y, Z_Ana, color="orange")
#w = ipv.plot_wireframe(X, Y, Z, color="red")
ipv.animation_control(s, add = True, interval=200)
#mylink = widgets.link((s, 'sequence_index'), (w, 'sequence_index'))
ipv.show()

In [ ]:
#Compare

In [ ]:
#Z_Num[0]-Z_Ana[0].reshape(1,-1)
print(u(0.90,0.75,0))
Z_Num[0][0]